## DENCE NET ARCHITECTURE

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import tensorflow
import tensorflow as tf
print(tf.__version__)

import keras
import keras.backend as K
from keras.models import Model
from keras.layers import Input, Dense, Conv2D, Conv3D, DepthwiseConv2D, SeparableConv2D, Conv3DTranspose
from keras.layers import Flatten, MaxPool2D, AvgPool2D, GlobalAvgPool2D, UpSampling2D, BatchNormalization
from keras.layers import Concatenate, Add, Dropout, ReLU, Lambda, Activation, LeakyReLU, PReLU

from IPython.display import SVG
from keras.utils import model_to_dot

from time import time
import numpy as np

from keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping

import warnings
warnings.filterwarnings('ignore')

2.13.0


In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train=ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True,validation_split = 0.2)
train_data=train.flow_from_directory(directory = 'DATASET/TRAIN',target_size=(224,224),
                                     batch_size=32,class_mode='categorical')

Found 2000 images belonging to 10 classes.


In [4]:
test=ImageDataGenerator(rescale=1./255)
test_data=test.flow_from_directory(directory = 'DATASET/TEST',target_size=(224,224),
                                   batch_size=32,class_mode='categorical')

Found 2000 images belonging to 10 classes.


In [5]:
def densenet(img_shape, n_classes, f=32):
  repetitions = 6, 12, 24, 16
  
  def bn_rl_conv(x, f, k=1, s=1, p='same'):
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Conv2D(f, k, strides=s, padding=p)(x)
    return x
  
  
  def dense_block(tensor, r):
    for _ in range(r):
      x = bn_rl_conv(tensor, 4*f)
      x = bn_rl_conv(x, f, 3)
      tensor = Concatenate()([tensor, x])
    return tensor
  
  
  def transition_block(x):
    x = bn_rl_conv(x, K.int_shape(x)[-1] // 2)
    x = AvgPool2D(2, strides=2, padding='same')(x)
    return x
  
  
  input = Input(img_shape)
  
  x = Conv2D(64, 7, strides=2, padding='same')(input)
  x = MaxPool2D(3, strides=2, padding='same')(x)
  
  for r in repetitions:
    d = dense_block(x, r)
    x = transition_block(d)
  
  x = GlobalAvgPool2D()(d)
  
  output = Dense(n_classes, activation='softmax')(x)
  
  model = Model(input, output)
  model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy',tensorflow.keras.metrics.Precision()])
  return model
     

input_shape = 224, 224, 3
n_classes = 10

K.clear_session()
model = densenet(input_shape, n_classes)
model.summary() 

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 112, 112, 64)         9472      ['input_1[0][0]']             
                                                                                                  
 max_pooling2d (MaxPooling2  (None, 56, 56, 64)           0         ['conv2d[0][0]']              
 D)                                                                                               
                                                                                                  
 batch_normalization (Batch  (None, 56, 56, 64)           256       ['max_pooling2d[0][0]']   

 )                                                                   'conv2d_8[0][0]']            
                                                                                                  
 batch_normalization_8 (Bat  (None, 56, 56, 192)          768       ['concatenate_3[0][0]']       
 chNormalization)                                                                                 
                                                                                                  
 re_lu_8 (ReLU)              (None, 56, 56, 192)          0         ['batch_normalization_8[0][0]'
                                                                    ]                             
                                                                                                  
 conv2d_9 (Conv2D)           (None, 56, 56, 128)          24704     ['re_lu_8[0][0]']             
                                                                                                  
 batch_nor

                                                                                                  
 conv2d_17 (Conv2D)          (None, 28, 28, 32)           36896     ['re_lu_16[0][0]']            
                                                                                                  
 concatenate_7 (Concatenate  (None, 28, 28, 192)          0         ['concatenate_6[0][0]',       
 )                                                                   'conv2d_17[0][0]']           
                                                                                                  
 batch_normalization_17 (Ba  (None, 28, 28, 192)          768       ['concatenate_7[0][0]']       
 tchNormalization)                                                                                
                                                                                                  
 re_lu_17 (ReLU)             (None, 28, 28, 192)          0         ['batch_normalization_17[0][0]
          

 tchNormalization)                                                                                
                                                                                                  
 re_lu_25 (ReLU)             (None, 28, 28, 320)          0         ['batch_normalization_25[0][0]
                                                                    ']                            
                                                                                                  
 conv2d_26 (Conv2D)          (None, 28, 28, 128)          41088     ['re_lu_25[0][0]']            
                                                                                                  
 batch_normalization_26 (Ba  (None, 28, 28, 128)          512       ['conv2d_26[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 re_lu_26 

 batch_normalization_34 (Ba  (None, 28, 28, 128)          512       ['conv2d_34[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 re_lu_34 (ReLU)             (None, 28, 28, 128)          0         ['batch_normalization_34[0][0]
                                                                    ']                            
                                                                                                  
 conv2d_35 (Conv2D)          (None, 28, 28, 32)           36896     ['re_lu_34[0][0]']            
                                                                                                  
 concatenate_16 (Concatenat  (None, 28, 28, 480)          0         ['concatenate_15[0][0]',      
 e)                                                                  'conv2d_35[0][0]']           
          

...

                                                                                                  
 conv2d_78 (Conv2D)          (None, 14, 14, 32)           36896     ['re_lu_77[0][0]']            
                                                                                                  
 concatenate_37 (Concatenat  (None, 14, 14, 896)          0         ['concatenate_36[0][0]',      
 e)                                                                  'conv2d_78[0][0]']           
                                                                                                  
 batch_normalization_78 (Ba  (None, 14, 14, 896)          3584      ['concatenate_37[0][0]']      
 tchNormalization)                                                                                
                                                                                                  
 re_lu_78 (ReLU)             (None, 14, 14, 896)          0         ['batch_normalization_78[0][0]
          

 tchNormalization)                                                                                
                                                                                                  
 re_lu_86 (ReLU)             (None, 14, 14, 1024)         0         ['batch_normalization_86[0][0]
                                                                    ']                            
                                                                                                  
 conv2d_87 (Conv2D)          (None, 14, 14, 512)          524800    ['re_lu_86[0][0]']            
                                                                                                  
 average_pooling2d_2 (Avera  (None, 7, 7, 512)            0         ['conv2d_87[0][0]']           
 gePooling2D)                                                                                     
                                                                                                  
 batch_nor

 concatenate_45 (Concatenat  (None, 7, 7, 640)            0         ['concatenate_44[0][0]',      
 e)                                                                  'conv2d_95[0][0]']           
                                                                                                  
 batch_normalization_95 (Ba  (None, 7, 7, 640)            2560      ['concatenate_45[0][0]']      
 tchNormalization)                                                                                
                                                                                                  
 re_lu_95 (ReLU)             (None, 7, 7, 640)            0         ['batch_normalization_95[0][0]
                                                                    ']                            
                                                                                                  
 conv2d_96 (Conv2D)          (None, 7, 7, 128)            82048     ['re_lu_95[0][0]']            
          

                                                                    ]']                           
                                                                                                  
 conv2d_104 (Conv2D)         (None, 7, 7, 128)            98432     ['re_lu_103[0][0]']           
                                                                                                  
 batch_normalization_104 (B  (None, 7, 7, 128)            512       ['conv2d_104[0][0]']          
 atchNormalization)                                                                               
                                                                                                  
 re_lu_104 (ReLU)            (None, 7, 7, 128)            0         ['batch_normalization_104[0][0
                                                                    ]']                           
                                                                                                  
 conv2d_10

 re_lu_112 (ReLU)            (None, 7, 7, 128)            0         ['batch_normalization_112[0][0
                                                                    ]']                           
                                                                                                  
 conv2d_113 (Conv2D)         (None, 7, 7, 32)             36896     ['re_lu_112[0][0]']           
                                                                                                  
 concatenate_54 (Concatenat  (None, 7, 7, 928)            0         ['concatenate_53[0][0]',      
 e)                                                                  'conv2d_113[0][0]']          
                                                                                                  
 batch_normalization_113 (B  (None, 7, 7, 928)            3712      ['concatenate_54[0][0]']      
 atchNormalization)                                                                               
          

In [6]:
model_path = "DENCE.h5"

from keras.callbacks import ModelCheckpoint

M = ModelCheckpoint(model_path, monitor='accuracy', verbose=1, save_best_only=True)

In [7]:
epochs = 50
batch_size = 256

In [ ]:
#### Fitting the model
history = model.fit(
           train_data, steps_per_epoch=train_data.samples // batch_size, 
           epochs=epochs, 
           validation_data=test_data,validation_steps=test_data.samples // batch_size,
           callbacks=[M])

Epoch 1/50
7/7 [==============================] - ETA: 0s - loss: 3.5549 - accuracy: 0.2946 - precision: 0.3969
Epoch 1: accuracy improved from -inf to 0.29464, saving model to DENCE.h5
7/7 [==============================] - 103s 11s/step - loss: 3.5549 - accuracy: 0.2946 - precision: 0.3969 - val_loss: 2.3119 - val_accuracy: 0.0759 - val_precision: 0.3846
Epoch 2/50
7/7 [==============================] - ETA: 0s - loss: 1.4876 - accuracy: 0.5670 - precision: 0.7402
Epoch 2: accuracy improved from 0.29464 to 0.56696, saving model to DENCE.h5
7/7 [==============================] - 67s 10s/step - loss: 1.4876 - accuracy: 0.5670 - precision: 0.7402 - val_loss: 3.0687 - val_accuracy: 0.1295 - val_precision: 0.3382
Epoch 3/50
7/7 [==============================] - ETA: 0s - loss: 1.5375 - accuracy: 0.5491 - precision: 0.6412
Epoch 3: accuracy did not improve from 0.56696
7/7 [==============================] - 65s 9s/step - loss: 1.5375 - accuracy: 0.5491 - precision: 0.6412 - val_loss: 2.54

In [ ]:
history.history.keys()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.figure(figsize=(20, 8))
plt.plot(history.history['accuracy'])

for i in range(epochs):
    if i%5 == 0:
        plt.annotate(np.round(history.history['accuracy'][i]*100,2),xy=(i,history.history['accuracy'][i]))

plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.show()

In [ ]:
plt.figure(figsize=(20, 8))
plt.plot(history.history['loss'])

for i in range(epochs):
    if i%5 == 0:
        plt.annotate(np.round(history.history['loss'][i]*100,2),xy=(i,history.history['loss'][i]))

plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.show()